In [4]:
import os

In [2]:
base_dir = "MOT20Dataset/train/MOT20-01"

In [7]:
# Need to convert MOT20 dataset to YOLO format
input_file = f'{base_dir}/gt/gt.txt'
output_dir = f'{base_dir}/labels'  # Directory to store YOLO annotations
image_width = 1920     
image_height = 1080

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Process gt.txt
with open(input_file, "r") as f:
    for line in f:
        # Split line and filter for required values
        parts = line.strip().split(",")
        
        # Ensure we have at least the first 7 values
        if len(parts) < 7:
            continue  # Skip invalid lines
        
        # Parse required values
        frame = int(parts[0])
        obj_id = int(parts[1])
        bb_left = float(parts[2])
        bb_top = float(parts[3])
        bb_width = float(parts[4])
        bb_height = float(parts[5])
        conf = float(parts[6])
        
        # Skip entries with conf == 0
        if conf == 0:
            continue

        # Normalize bounding box coordinates
        x_center = (bb_left + bb_width / 2) / image_width
        y_center = (bb_top + bb_height / 2) / image_height
        width = bb_width / image_width
        height = bb_height / image_height

        # YOLO format: <class_id> <x_center> <y_center> <width> <height>
        yolo_line = f"{int(obj_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

        # Write to frame-specific file
        output_file = os.path.join(output_dir, f"{int(frame):06d}.txt")
        with open(output_file, "a") as out_f:
            out_f.write(yolo_line + "\n")